In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns

In [2]:
guests = pd.read_excel("data/guests.xlsx", index_col=0)
guests.drop("guest", axis= 1, inplace = True)
guests = 1 - guests 
vec_discount = guests["discount"].values #converts the discount column to a vector

In [3]:
hotels = pd.read_excel("data/hotels.xlsx", index_col=0)
vec_prices = hotels["price"].values

In [4]:
preferences = pd.read_excel("data/preferences.xlsx", index_col=0)
preferences["guest"] = pd.to_numeric(preferences["guest"].str.replace('guest_','')) - 1
preferences["hotel"] = pd.to_numeric(preferences["hotel"].str.replace('hotel_','')) - 1
pref_pivoted = pd.pivot_table(preferences, index ='guest', columns ='hotel', values= "priority" , aggfunc="min") #it pivots the original dataframe to get to a matrix-like format
rank_matrix = pref_pivoted.rank(axis = 1, method="dense").values #reorders preferences

In [5]:
#This function creates a utility matrix from the ranked preferences matrix
def to_utility_matrix(rank_matrix):
  user_utility = []
  for row in rank_matrix:
    utility_values = np.linspace(start=1, stop= 0.1, num=len(set(row[~np.isnan(row)]))) #utility goes from 1 to 0.1 split evenly between preferences
    user_utility.append([utility_values[int(i)-1] if i == i else i for i in row ]) #i == i to check if i is not nan 
  return np.nan_to_num(np.array(user_utility), nan = 0)

In [6]:
utility_matrix = to_utility_matrix(rank_matrix)

In [7]:
# def print_output(res, guest_count, room_count, hotel_count, t):
#   res_avg = np.round_(np.array(res).mean(axis=0), 3)
#   print(f"With {t} iterations, on average we have: \n{res_avg[0]}/{guest_count} guests placed \n{res_avg[1]}/{room_count} rooms occupied \n{res_avg[2]}/{hotel_count} hotels occupied \n{res_avg[3]} total revenue \n{res_avg[4]} total satisfaction")

In [8]:
def results(choice_matrix, vec_prices=vec_prices, vec_discount=vec_discount, utility_matrix=utility_matrix):
  guests_placed = choice_matrix.sum(axis=1).sum()
  rooms_occupied = choice_matrix.sum(axis=0).sum()
  hotels_occupied = np.where(choice_matrix.sum(axis=0) > 0, 1, 0).sum()
  prices_matrix = vec_prices * choice_matrix #all 1s in the matrix will be replaced by that hotel's price
  discount_matrix = np.multiply(prices_matrix, vec_discount[:, np.newaxis]) #the room price is then discounted for each user
  revenue = discount_matrix.sum()
  utility = np.multiply(choice_matrix, utility_matrix).sum()
  return([guests_placed, rooms_occupied, hotels_occupied, revenue, utility])

In [9]:
def random_model(hotels, vec_prices, vec_discount, rank_matrix, t=1000):
  guest_count, room_count, hotel_count = rank_matrix.shape[0] , hotels["rooms"].sum(), rank_matrix.shape[1]
  least = min([guest_count, room_count])
  res = []
  vec_rooms = np.repeat(hotels["price"], hotels["rooms"]).index.values #Denormalized the dataframe by adding rows. This vec contains the hotel number as many times as its rooms 
  for _ in range(t):
    choice_matrix = np.zeros(rank_matrix.shape) #creates filled with zeros 4000 x 400
    random_rooms = np.random.choice(vec_rooms, least, replace=False ) #selects randomly 4000 rooms from all possible rooms
    choice_matrix[tuple(range(least)),tuple(random_rooms)] = 1 #assign 1 to all occupied rooms given their indices
    res.append(results(choice_matrix, vec_prices, vec_discount))
    #list(np.array(res).mean(axis=0))
  return np.array(res)

In [10]:
def preference_model(hotels, vec_prices, rank_matrix):
  rank_matrix_filled, choice_matrix  = np.nan_to_num(rank_matrix), np.zeros(rank_matrix.shape) 
  rooms = hotels["rooms"].values
  for idx, row in enumerate(rank_matrix_filled):
    c = np.multiply(row, np.where(rooms > 0, 1, 0) ) #sets preferences values to 0 where there is no room availability
    try:
      user_choice = np.where(c == np.min(c[np.nonzero(c)]), 1, 0) #The minimum nonzero is converted to 1 and the rest to 0
      choice_matrix[(idx),] = user_choice
      rooms = rooms - user_choice #subtract 1 from the room availability where it's now occupied 
    except ValueError: #pass if array are all zeros
     pass
  return  results(choice_matrix)

In [11]:
def room_model(hotels,rank_matrix):

    pref_pivoted = pd.DataFrame(rank_matrix).T
    pref_pivoted["price"] = hotels["price"]
    pref_pivoted["rooms"] =  hotels["rooms"] #add columns with number of rooms and price per hotel
    h_rooms = pref_pivoted.sort_values(by=['rooms', 'price'], ascending=[False, True]) # DF sorted by number of rooms and price
    h_rooms = h_rooms.drop(columns=["rooms","price"]) # drop columns 'rooms' and 'price'

    choice_matrix = np.zeros(rank_matrix.shape) 
    rooms = hotels["rooms"].values # available rooms per hotel

    for h, rows in h_rooms.iterrows():                          
        df_pref = pd.DataFrame(rows.values, columns=['values']) # for each row in h_rooms: create DF with row values (preferences) and index
        df_pref['index'] = df_pref.index.values 
        sorted_guests = df_pref.dropna().sort_values(by=['values','index'])['index'] # dropna, sort DF and consider 'index' of the remaining guests
        selected_guests = sorted_guests[:rooms[h]].values # select as many guests as possible (maximum number: rooms[h])
        
        for guest in selected_guests:
            choice_matrix[guest][h] = 1 # add selected guests to the choice_matrix
            #h_rooms.drop([idx], axis=1) ??????
            h_rooms.iloc[:][guest]= np.nan # for each selected guest transform preference values in np.nan 
        
        
        
        
    return  results(choice_matrix)

In [ ]:
def price_model(hotels, rank_matrix):
    pref_pivoted = pd.DataFrame(rank_matrix).T 
    pref_pivoted["price"] = hotels["price"] #add a column with prices per hotel
    pref_pivoted["rooms"] = hotels["rooms"] #add a column with number of rooms per hotel
    pref_by_price = pref_pivoted.sort_values(by=['price', 'rooms'], ascending=[True, False]) #sorted by price and more rooms
    pref_by_price = pref_by_price.drop(columns=["rooms","price"]) #drop columns: price and rooms
    choice_matrix = np.zeros(rank_matrix.shape) #define matrix with zeros, same shape as rank_matrix
    free_rooms = hotels["rooms"].values #vector of rooms from dataset hotels
    
    for i, rows in pref_by_price.iterrows(): #for hotel with name i, consider the row of guests
        hotel_i = pd.DataFrame(rows.values, columns=['values']) #transforming the row in dataframe 
        hotel_i['index'] = hotel_i.index.values #consider the index of values
        hotel_i = hotel_i.dropna().sort_values(by=['values','index'])['index'] #soting
        guest4hotel_i = hotel_i[:free_rooms[i]].values #take values of guests for free rooms
        for name_guest in guest4hotel_i:
            choice_matrix[name_guest][i] = 1
            pref_by_price.iloc[:][name_guest]= np.nan
            
    return results(choice_matrix)

In [ ]:
random_results_array = random_model(hotels, vec_prices, vec_discount, rank_matrix, t=1000) #return the results of random model in array

In [ ]:
#preference_model(hotels, vec_prices, rank_matrix)

In [12]:
room_model(hotels, rank_matrix)

[4000.0, 4000.0, 341, 633529.3699999999, 3679.8077205786803]

In [ ]:
#price_model(hotels, rank_matrix)

In [ ]:
#We memorize the results in variables and then put them into the matrix
#random_model_results = random_results_array.mean(axis=0)
#preference_model_results = preference_model(hotels, vec_prices, rank_matrix)
#room_model_results = room_model(hotels, rank_matrix)
#price_model_results = price_model(hotels, rank_matrix)
#matrix_results = np.array([random_model_results, preference_model_results, room_model_results, price_model_results])

In [ ]:
matrix_results = np.array([random_results_array.mean(axis=0), preference_model(hotels, vec_prices, rank_matrix), room_model(hotels, rank_matrix), price_model(hotels, rank_matrix)])
matrix_results

In [ ]:
#Define a dataframe with matrix_results
df_matrix_results = pd.DataFrame(matrix_results, index = ['Random Model', 'Preference Model', 'Availability Model', 'Price Model'], columns = ['Number of Clients arranged','Number of Rooms occupied','Number of Hotels','Revenue', 'Degree of satisfaction'])
df_matrix_results.astype(int)

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=5, figsize=(20, 4))
x = ['Random', 'Pref', 'Rooms', 'Price']
axes[0].bar(x, matrix_results[:,0], color = sns.color_palette("viridis"))
axes[0].set_title("Number of clients")
axes[0].set_ylim(3900,4050)
axes[1].bar(x, matrix_results[:,1], color = sns.color_palette("rocket"))
axes[1].set_title("Number of rooms")
axes[1].set_ylim(3900,4050)
axes[2].bar(x, matrix_results[:,2], color = sns.color_palette("magma"))
axes[2].set_title("Number of hotels")
axes[2].set_ylim(300,410)
axes[3].bar(x, matrix_results[:,3], color = sns.color_palette("flare"))
axes[3].set_title("Revenue")
axes[3].set_ylim(570000,650000)
axes[4].bar(x, matrix_results[:,4], color = sns.color_palette("mako"))
axes[4].set_title("Degree of satisfaction")
axes[4].set_ylim(100,4050)

In [ ]:
#fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(15, 5)) #value of array defined before
#axes[0].boxplot([random_results_array[:,3], preference_model_results[3], room_model_results[3], price_model_results[3]])
#axes[0].set_title("Revenue")
#axes[1].boxplot([random_results_array[:,4], preference_model_results[4], room_model_results[4], price_model_results[4]])
#axes[1].set_title("Degree of satisfaction")

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(15, 5)) #values of matrix_results
axes[0].boxplot([random_results_array[:,3], matrix_results[1, 3], matrix_results[2, 3], matrix_results[3, 3]])
axes[0].set_title("Revenue")
axes[1].boxplot([random_results_array[:,4], matrix_results[1, 4], matrix_results[2, 4], matrix_results[3, 4]])
axes[1].set_title("Degree of satisfaction")

In [ ]:
pd.DataFrame(random_results_array, columns = ['Number of Clients arranged','Number of Rooms occupied','Number of Hotels','Revenue', 'Degree of satisfaction']).describe()

In [ ]:
sns.histplot(random_results_array[:,3], bins = 50, color = 'darkcyan') #plot an histogram - Revenue for Random Model

In [ ]:
sns.histplot(random_results_array[:,4], color = 'darkcyan') #plot an histogram - Degree of satisfaction for Random Model   